**Install Packages**

In [ ]:
!pip install pycaret
!pip install mysql-connector-python
!pip install sqlalchemy==1.4.32
!pip install MLB-StatsAPI

**Import Packages**

In [55]:
import pycaret
import pandas as pd
import numpy as np
import sqlalchemy
import mysql.connector
import sys
import requests
import statsapi
from constants import API_KEY, database_name, sql_hostname

from pycaret import classification
from datetime import datetime
# from google.colab.data_table import DataTable
# from google.colab import files
# from google.colab import drive

# sys.path.append("/content/drive/MyDrive")
#
# drive.mount('/content/drive')

def odds_calculator(probability):

    return round(-100 / ((1/probability)-1))

**Import Data**

In [56]:

engine = sqlalchemy.create_engine(f'{sql_hostname}/{database_name}')

Raw_Dataset = pd.read_sql(sql = "SELECT * FROM baseball_spread_production", con = engine).set_index("game_datetime").reset_index()[["game_datetime", "team_1",	"team_1_spread_odds",	"team_2",	"team_2_spread_odds",	"venue_name"]]

print(len(Raw_Dataset))

5


In [57]:
Raw_Dataset

,game_datetime,team_1,team_1_spread_odds,team_2,team_2_spread_odds,venue_name
0,2023-08-13 16:05:00,Boston Red Sox,-115,Detroit Tigers,-115,Fenway Park
1,2023-08-13 17:35:00,Washington Nationals,130,Oakland Athletics,-166,Nationals Park
2,2023-08-13 17:40:00,New York Yankees,-120,Miami Marlins,-110,loanDepot park
3,2023-08-13 20:10:00,Los Angeles Dodgers,-125,Colorado Rockies,-105,Dodger Stadium
4,2023-08-13 23:10:00,Atlanta Braves,114,New York Mets,-135,Citi Field


### **Load Models**

In [58]:
#Finalized_Model_save_to_file_string = f"2023-07-22 Baseball Spread"
Finalized_Model_save_to_file_string = f"{datetime.today().strftime('%Y-%m-%d')}_Baseball_Spread_model_nb"

Classification_Model = pycaret.classification.load_model(f"./models/{Finalized_Model_save_to_file_string}")

print(Finalized_Model_save_to_file_string)

Transformation Pipeline and Model Successfully Loaded
2023-08-13_Baseball_Spread_model_nb


### **Get Predictions of Day**

In [59]:
Prediction = pycaret.classification.predict_model(estimator = Classification_Model, data = Raw_Dataset)

In [60]:
Prediction["theo_odds"] = Prediction["prediction_score"].apply(odds_calculator)

In [61]:
Prediction[["team_1", "team_2","prediction_label","prediction_score", "theo_odds"]].sort_values(by = "prediction_score", ascending = False)

,team_1,team_2,prediction_label,prediction_score,theo_odds
3,Los Angeles Dodgers,Colorado Rockies,1,0.7972,-393
0,Boston Red Sox,Detroit Tigers,1,0.7519,-303
4,Atlanta Braves,New York Mets,0,0.6721,-205
2,New York Yankees,Miami Marlins,1,0.6465,-183
1,Washington Nationals,Oakland Athletics,1,0.5233,-110
